In [ ]:
import setup

setup.init_django()

In [ ]:
from decouple import config

In [ ]:
from rag import db as rag_db, settings as rag_settings, engines as rag_engines

In [ ]:
rag_settings.init()
semantic_query_engine = rag_engines.get_semantic_query_engine()
sql_query_engine = rag_engines.get_sql_query_engine(sql_only=True)

In [ ]:
print_default_prompts = True

In [ ]:
prompts_dict = sql_query_engine.get_prompts()

if print_default_prompts:
    print(prompts_dict['response_synthesis_prompt'].template)

In [ ]:
if print_default_prompts:
    print(sql_query_engine.sql_retriever.get_prompts()['text_to_sql_prompt'].template)

In [ ]:
from llama_index.core import PromptTemplate

In [ ]:
custom_sql_prompt = PromptTemplate("""Given an input question, create a precise {dialect} PostgreSQL query to answer it. Follow these guidelines:

1. Use only tables and columns from the provided schema.
2. Select only relevant columns, never all columns.
3. Qualify column names with table names when necessary.
4. Use appropriate JOINs, WHERE clauses, and aggregations.
5. Order results to highlight the most pertinent information.
6. Avoid querying non-existent columns or tables.
7. Optimize the query for performance where possible.

Your response should contain only the SQL query, without any additional explanation or formatting. Do not use markdown or prepend the query with the term `sql`.

Schema:
{schema}

Question: {query_str}
SQL Query:
""")

In [ ]:
custom_response_synthesis_prompt = PromptTemplate("""Given an input question, synthesize a response from the query results.
Query: {query_str}
SQL: {sql_query}
SQL Response: {context_str}
Response: 
""")

In [ ]:
custom_response_synthesis_prompt.template_vars

In [ ]:
sql_query_engine = rag_engines.get_sql_query_engine(
    sql_only=False,
    synthesize_response=True
)

In [ ]:
# print(sql_query_engine.sql_retriever.get_prompts()['text_to_sql_prompt'].template)

In [ ]:
response = sql_query_engine.query("What blog post has the most views from today?")
print(str(response))

In [ ]:
for node in response.source_nodes:
    print(node.node.get_content())